In [3]:
import base64
import json
import requests
import traceback
from os import path
from requests.exceptions import HTTPError

In [2]:
dbutils.widgets.removeAll()

NameError: name 'dbutils' is not defined

In [0]:
dbutils.widgets.text("job_unique_name", "job_silver_sales_global_nielsen_per")
job_unique_name = dbutils.widgets.get("job_unique_name")
print(f"job_unique_name: {job_unique_name}")

dbutils.widgets.text("notebook_language", "PYTHON")
notebook_language = dbutils.widgets.get("notebook_language").upper()
print(f"notebook_language: {notebook_language}")

dbutils.widgets.text("notebook_code", "print('Hello, World')")
notebook_code = dbutils.widgets.get("notebook_code")
print(f"notebook_code: {notebook_code}")

dbutils.widgets.text("watermark_start_datetime", "2021-12-31 00:00:00")
watermark_start_datetime = dbutils.widgets.get("watermark_start_datetime")
print(f"watermark_start_datetime: {watermark_start_datetime}")

dbutils.widgets.text("watermark_end_datetime", "2021-12-31 23:59:59")
watermark_end_datetime = dbutils.widgets.get("watermark_end_datetime")
print(f"watermark_end_datetime: {watermark_end_datetime}")

job_unique_name: job_silver_sales_global_nielsen_per
notebook_language: PYTHON
notebook_code: print('Hello, World')
start_datetime: 2021-12-31 00:00:00
end_datetime: 2021-12-31 23:59:59

In [0]:
try:
    # Get API details
    context = json.loads(dbutils.notebook.entry_point.getDbutils().notebook().getContext().toJson())
    api_url = context["extraContext"]["api_url"]
    api_token = context["extraContext"]["api_token"]
    notebook_path = context["extraContext"]["notebook_path"]
    new_notebook_path = path.join(path.dirname(notebook_path), "job_notebooks", job_unique_name)
    notebook_content_base64 = base64.b64encode(notebook_code.encode("ascii")).decode("ascii")

    payload = {
        "path": new_notebook_path,
        "format": "SOURCE",
        "language": notebook_language,
        "content": notebook_content_base64,
        "overwrite": True,
    }

    headers = {
        "Authorization": f"Bearer {api_token}",
        "Content-Type": "application/json",
    }

    # Deploy notebook using REST API
    response = requests.post(
        f"{api_url}/api/2.0/workspace/import",
        json=payload,
        headers=headers,
    )

    # Throw error if status was not OK
    if not response:
        raise HTTPError(response.status_code, response.json())

    # Run notebook
    timeout_seconds = 3*60*60  # 3 hours
    params = {
        "watermark_start_datetime": watermark_start_datetime,
        "watermark_end_datetime": watermark_end_datetime,
    }
    resultsJson = dbutils.notebook.run(new_notebook_path, timeout_seconds, params)

    # Throw error if result was not provided
    if not resultsJson:
        raise ValueError("Child notebook has not returned any result. Did you forget to call the framework's exit function?")

    results = json.loads(resultsJson)

except Exception as e:
    results = {
        "status": "FAILED",
        "target_object": None,
        "num_records_read": None,
        "num_records_loaded": None,
        "num_records_errored_out": None,
        "error_message": str(e)[:8000],
        "error_details": traceback.format_exc()
    }

print(results)

{'status': 'FAILED', 'error_message': "Child notebook has not returned any result. Did you forget to call the framework's exit function?", 'detailed_error_message': 'Traceback (most recent call last):\n File "<command-3198563744359297>", line 40, in <module>\n raise ValueError("Child notebook has not returned any result. Did you forget to call the framework\'s exit function?")\nValueError: Child notebook has not returned any result. Did you forget to call the framework\'s exit function?\n'}

In [0]:
dbutils.notebook.exit(json.dumps(results))

{"status": "FAILED", "error_message": "Child notebook has not returned any result. Did you forget to call the framework's exit function?", "detailed_error_message": "Traceback (most recent call last):\n  File \"<command-3198563744359297>\", line 40, in <module>\n    raise ValueError(\"Child notebook has not returned any result. Did you forget to call the framework's exit function?\")\nValueError: Child notebook has not returned any result. Did you forget to call the framework's exit function?\n"}